In [1]:
import os
os.chdir('..')

from util import CONFIG
CONFIG.set_use_mpl_tables(True)

In [2]:
from math import log2
import numpy as np

from sim_circuit import QuantumRegister, QuantumCircuit


def phase_estimation_unitary(n, U, swap=True):
    assert(U.shape[0] == U.shape[1])
    m = int(log2(U.shape[0]))

    eigvals, eigvecs = np.linalg.eig(U)

    q = QuantumRegister(n)
    a = QuantumRegister(m)
    qc = QuantumCircuit(q, a) # ancilla is last

    qc.append_u(eigvecs, a)
    qc.report('eigenstate')

    for i in range(n):
        qc.h(q[i])

    for i in range(n):
        if swap:
            for _ in range(2**i):
                qc.c_append_u(U, q[i], a)
        else:
            # qubit reversal
            for _ in range(2**i):
                qc.c_append_u(U, q[n-1-i], a)
            # decreasing powers of 2
            # for _ in range(2**(n-1-i)):
            #     qc.c_append_u(U, q[i], a)

    qc.report('geometric_sequence_superposition')

    qc.append_u(np.conj(eigvecs.transpose()), a)

    qc.report('geometric_sequence')

    qc.iqft(q if swap else q[::-1], swap)
    qc.report('estimate')

    return qc

In [3]:
    import scipy.stats
    
    n=3
    m = 2
    
    U = scipy.stats.unitary_group.rvs(2**m)
    qc = phase_estimation_unitary(n, U, swap=True)

In [4]:
from math import pi

eigvals, _ = np.linalg.eig(U)
theta = np.angle(eigvals[0])
if theta < 0:
    theta += 2*pi

v = theta/pi*2**(n-1)
print('\nfrequency from eigenvalue', v)


frequency from eigenvalue 3.233494306415035


In [5]:
from util import print_state_table

state = qc.run()
print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        00000   0.0753 - i0.0448    0.0876      -30.25°   ██                        0.0077
1        00001   0.1077 - i0.0156    0.1088       -8.76°   ██                        0.0118
2        00010   0.1742 + i0.0443    0.1797       14.27°   ████                      0.0323
3        00011   0.7321 + i0.5472    0.914        36.78°   █████████████████████     0.8354
4        00100  -0.1442 - i0.2427    0.2823     -120.28°   ██████                    0.0797
5        00101  -0.0187 - i0.1295    0.1309      -98.78°   ███                       0.0171
6        00110   0.0233 - i0.0916    0.0946      -75.27°   ██                        0.0089
7        00111   0.0503 - i0.0673    0.084       -53.77°   ██                        0.0071
8        01000   0.0000 + i0.0000    0.0                             

In [6]:
from util import all_close, cis, prod
from math import cos

def complex_sincd(n, v):
    N = 2 ** n
    return [prod(
        cos((v - k) * pi / 2 ** (j + 1)) * cis((v - k) * pi / 2 ** (j + 1))
        for j in range(n)) for k in range(2 ** n)]


assert all_close(state, complex_sincd(n, v))

In [7]:
def test_unitary_inverse():
    n = 3
    m = 2
    
    U = scipy.stats.unitary_group.rvs(2**m)
    qc = phase_estimation_unitary(n, U, swap=True)
    qci = qc.inverse()

    qc.append(qci, QuantumRegister(m+n))
    state = qc.run()
    tabulate_state(state)

    assert all_close(state, init_state(m+n))